In [65]:
import sys, pygame
from scipy.spatial import distance
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy import array
import math
from sklearn.preprocessing import normalize
#w asal impor aja ini, kali aja scipy misal punya tools kaya math, tapi yasudahlah
from random import randint

pygame.init()

#inisialisasi training parameter dan game
size = width, height = 480, 520
black = 255,255,255
ball_speed = [2,2]
block_speed = [randint(-2,2),0]
training_move = [0]
training_distance = []
training_velocity = []
training = np.zeros((1,8,1))


In [66]:
#buat model
class Model:
    model = None
    def __init__(self,shape=(8,1),hidden_layer=30,output_layer=5):
        self.model = keras.Sequential([
            keras.layers.Flatten(input_shape=shape),
            keras.layers.Dense(hidden_layer, activation=tf.nn.relu),
            keras.layers.Dense(output_layer, activation=tf.nn.softmax)
        ])
    def Comp(self):    
        self.model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return self.model
    def Fit(self,training,training_move,epoch=50):
        self.model.fit(training, training_move, epochs=epoch)
        test_loss, test_acc = self.model.evaluate(training, training_move)

        print('Test accuracy:', test_acc)
        
    def Predict(self,c):
        predictions= self.model.predict(c)
        block_speed = (np.argmax(predictions[0])*4)-2
        return block_speed

In [ ]:
#tes ukuran tensor
a=np.zeros((2,3,1))
x=np.zeros((2))
b=np.array([[1],[1],[1]])
a=np.insert(training, len(training), b, axis=0)
a=np.insert(training, len(training), b, axis=0)
tes = []
tes.append(1)
print(len(tes))
print(a)
print(a.shape)

In [67]:
#training degea dengan algoritma biasa, dengan melihat perbedaan jarak dengan bola

ball = pygame.image.load("ball.png")
ball = pygame.transform.scale(ball,(width//32,height//32))
ballrect = ball.get_rect().move([randint(0,width),randint(10,height)])

block = pygame.image.load("degea.png")
block = pygame.transform.scale(block,(width//8,height//8))
blockrect = block.get_rect().move([width//2,0])

field = pygame.image.load("field.jpg")
field = pygame.transform.scale(field,(width,height))

range_ball = 0
score = 0
award = 0
touch = False
hit_wall = 0
model = Model()
model.Comp()

screen = pygame.display.set_mode(size)
pygame.display.set_caption("De Gea Catching Ball")
while 1:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            model.Fit(training,training_move)
            sys.exit()
    if touch==True:
        model.Fit(training,training_move)
        sys.exit()

    ballrect = ballrect.move(ball_speed)
    blockrect = blockrect.move(block_speed)
    
    ballx = (ballrect.right+ballrect.left)//2
    bally = (ballrect.top+ballrect.bottom)//2
    blockx = (blockrect.right+blockrect.left)//2
    blocky = (blockrect.top+blockrect.bottom)//2
    dx=ballx-blockx
    dy=bally-blocky
    
    dxn = dx / np.linalg.norm(width)
    dyn = dy / np.linalg.norm(height)
    
    range_ball = distance.euclidean((ballx,bally),(blockx,blocky))
    
    if ballrect.left < 0 or ballrect.right > width:
        ball_speed[0] = -ball_speed[0]
    if ballrect.top < 0:
        ball_speed[1] = -ball_speed[1]
        award += -10
        touch=True
    if ballrect.bottom > height:
        ball_speed[1] = -ball_speed[1]
    if (bally < (blockrect.bottom+blockrect.top)//2) and (ballx < blockrect.right) and (ballx > blockrect.left):
        ball_speed[1] = -ball_speed[1]
        score += 10
        award += 10
    if (bally < (blockrect.bottom+blockrect.top)//2) and (ballx > blockrect.left and ballx< blockrect.right):
        ball_speed[0] = -ball_speed[0]
        score += 10
        award += 10
    
    if blockrect.left < 0 or blockrect.right > width:
        block_speed[0] = -block_speed[0]
    else:
        if(ballx-blockx)<0: 
            if range_ball<400:
                block_speed[0] = -2
            else:
                block_speed[0] = -1
        if(ballx-blockx)>0: 
            if range_ball<400:
                block_speed[0] = 2
            else:
                block_speed[0] = 1
        #block_speed = [randint(-2,2),0]
    
   
    font = pygame.font.Font(None, 20)
    text = font.render("fail:"+str(touch)+"Cikk "+str(score)+" Award:"+str(award) + " touch:"+"{0:.2f}".format(range_ball), 1, (10, 10, 10)) # buat cek +" posisi bola:"+str((ballrect.right+ballrect.left)/2)+ " posisi meja:"+str(blockrect.right)
    textpos = text.get_rect(centery=screen.get_height()-10) #centerx=screen.get_width()//2
    
    leftwall=ballx
    rightwall=width-ballx
    topwall=bally
    if((ball_speed[0]<0 and leftwall<topwall)or(ball_speed[0]>0 and rightwall<topwall)):
        hit_wall=1
    else:
        hit_wall=0
        
    #training_distance.append(range_ball)
    #trainging_velocity.append(ball_speed)
    a=(ballrect[0]+1)/2
    b=(ballrect[1]+1)/2
    c=[[range_ball],[a],[b],[dxn],[dyn],[hit_wall],[ball_speed[0]],[blockrect[0]]]
    d=(block_speed[0]+2)/4 #np.linalg.norm(width)
    
    
    if(score>500): 
        ball_speed[0]+(ball_speed[0]/abs(ball_speed[0]))
        ball_speed[1]+(ball_speed[1]/abs(ball_speed[1]))
    training=np.insert(training, len(training), c, axis=0)
    training_move.append(d)
    
    screen.fill(black)
    screen.blit(field, [0,0])
    screen.blit(text, textpos)
    screen.blit(ball, ballrect)
    screen.blit(block, blockrect)
    pygame.display.flip()

Epoch 1/50
33845/33845 [==============================] - 2s 73us/step - loss: 8.0878 - acc: 0.4981
Epoch 2/50
33845/33845 [==============================] - 2s 50us/step - loss: 8.0874 - acc: 0.4982
Epoch 3/50
33845/33845 [==============================] - 2s 62us/step - loss: 8.0874 - acc: 0.4982
Epoch 4/50
33845/33845 [==============================] - 2s 57us/step - loss: 8.0874 - acc: 0.4982
Epoch 5/50
33845/33845 [==============================] - 2s 55us/step - loss: 8.0874 - acc: 0.4982
Epoch 6/50
33845/33845 [==============================] - 2s 50us/step - loss: 8.0874 - acc: 0.4982: 0s - loss: 8.0714 - 
Epoch 7/50
33845/33845 [==============================] - 2s 51us/step - loss: 8.0874 - acc: 0.4982
Epoch 8/50
33845/33845 [==============================] - 2s 55us/step - loss: 8.0874 - acc: 0.4982
Epoch 9/50
33845/33845 [==============================] - 2s 56us/step - loss: 8.0874 - acc: 0.4982
Epoch 10/50
33845/33845 [==============================] - 2s 56us/step - loss

SystemExit: 

In [27]:
model1=model

Tes hasil training so far--> 60365 data (2500 score), 100 epoch, 81%
--> 64973 data (2680 score), 50 epoch, 84%
--> 33845 data (1400 score), 50 epoch, 95%


In [70]:
# test game dengan arsitektur network yang telah ditrain

screen = pygame.display.set_mode(size)

block_speed = [randint(-2,2),0]
ball_speed = [2,2]
ballrect = ball.get_rect().move([randint(0,width),randint(10,height)])
blockrect = block.get_rect().move([width//2,0])

range_ball = 0
score = 0
award = 0
touch = False
hit_wall = False


while 1:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            sys.exit()
    if touch==True:
        print("ouch")
        sys.exit()

    ballrect = ballrect.move(ball_speed)
    blockrect = blockrect.move(block_speed)
    
    ballx = (ballrect.right+ballrect.left)//2
    bally = (ballrect.top+ballrect.bottom)//2
    blockx = (blockrect.right+blockrect.left)//2
    blocky = (blockrect.top+blockrect.bottom)//2
    dx=ballx-blockx
    dy=bally-blocky
    
    dxn = dx / np.linalg.norm(width)
    dyn = dy / np.linalg.norm(height)
    
    range_ball = distance.euclidean((ballx,bally),(blockx,blocky))
    
    if ballrect.left < 0 or ballrect.right > width:
        ball_speed[0] = -ball_speed[0]
    if ballrect.top < 0:
        ball_speed[1] = -ball_speed[1]
        award += -10
        touch=True
    if ballrect.bottom > height:
        ball_speed[1] = -ball_speed[1]
    if (bally < (blockrect.bottom+blockrect.top)//2) and (ballx < blockrect.right) and (ballx > blockrect.left):
        ball_speed[1] = -ball_speed[1]
        score += 10
        award += 10
    if (bally < (blockrect.bottom+blockrect.top)//2) and (ballx > blockrect.left and ballx< blockrect.right):
        ball_speed[0] = -ball_speed[0]
        score += 10
        award += 10
        
   
    font = pygame.font.Font(None, 20)
    text = font.render("fail:"+str(touch)+"Cikk "+str(score)+" Award:"+str(award) + " touch:"+"{0:.2f}".format(range_ball), 1, (10, 10, 10)) # buat cek +" posisi bola:"+str((ballrect.right+ballrect.left)/2)+ " posisi meja:"+str(blockrect.right)
    textpos = text.get_rect(centery=screen.get_height()-10) #centerx=screen.get_width()//2
    
    leftwall=ballx
    rightwall=width-ballx
    topwall=bally
    if((ball_speed[0]<0 and leftwall<topwall)or(ball_speed[0]>0 and rightwall<topwall)):
        hit_wall=1
    else:
        hit_wall=0
    
    a=(ballrect[0]+1)/2
    b=(ballrect[1]+1)/2
    c=np.array([([range_ball],[a],[b],[dxn],[dyn],[hit_wall],[ball_speed[0]],[blockrect[0]])])
    
    block_speed[0]=model.Predict(c)
    print(block_speed[0])
    
    screen.fill(black)
    screen.blit(field, [0,0])
    screen.blit(text, textpos)
    screen.blit(ball, ballrect)
    screen.blit(block, blockrect)
    pygame.display.flip()

-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
2
2
2
2
2
2
2
2
-2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
2
-2
-2
-2
2
-2
-2
2
-2
-2
2
-2
-2
2
-2
2
-2
-2
2
2
-2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
2
-2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2

-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
-2
-2
2
-2
2
2
-2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-

-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
-2
2
-2
2
2
-2
-2
-2
2
-2
2
2
-2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
2
-2
-2
2
-2
-2
2
-2
2
-2
-2
2
2
-2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-

-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
2
-2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-

-2
2
-2
2
-2
-2
-2
2
-2
2
2
-2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
-2
2
-2
2
2
-2
-2
-2
2
-2
2
2
-2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2

...

-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
-2
2
-2
2
2
-2
-2
-2
2
-2
2
2
-2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
2
-2
-2
2
-2
-2
2
-2
2
-2
-2
2
2
-2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
2
-2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
2
-2
-2
-2
2
-2
2
2
-2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
2
-2
2
-2
2
2
-2
-2
-2
2
-2
2
2
-2
2
2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
2
-2
-2
2
-2
-2
2
-2
2
-2
-2
2
2
-2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
-2
-2
2
-2
-2
-2
-2
-2
2
-2
-2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2
-2

SystemExit: 